In [11]:
import numpy as np 
import pandas as pd
import networkx as nx
import matplotlib as plt 
import markov_clustering as mc
import random
import csv

In [2]:
run rome.py

In [3]:
data = pd.read_csv("../bases/rome_tension.csv")

In [4]:
data = data[['rome5','BE_id','T']]
data = data.pivot(index='BE_id',columns='rome5',values='T')

In [5]:
romes = list(graph.keys())

### Undirected graph

In [26]:
G = nx.Graph()
G.add_nodes_from(romes)
weights = {}
R = 1
for rome1 in romes:
    weights[rome1] = {}
    s = 0
    for rome2 in graph[rome1]:
        if True: # replace by rome1[0] = rome2[0]
            weights[rome1][rome2] = np.float_power(data[rome1].corr(data[rome2]),R)
            s += weights[rome1][rome2]
    for rome2 in graph[rome1]:
        if True: # replace by rome1[0] = rome2[0]
             weights[rome1][rome2] =  weights[rome1][rome2]/s

for rome1 in romes:
    current_edges = list(G.edges)
    for rome2 in graph[rome1]:
        if True: # replace by rome1[0] = rome2[0]
            if rome1 not in graph[rome2]: w = weights[rome1][rome2]/2
            if rome1 in graph[rome2]: w = (weights[rome1][rome2]+weights[rome2][rome1])
            if (rome2,rome1) not in current_edges:
                G.add_edge(rome1,rome2,weight=w)
                
matrix = nx.adjacency_matrix(G)
result = mc.run_mcl(matrix,inflation=2.0)
clusters = mc.get_clusters(result)
len(clusters)

225

In [27]:
CL = [[romes[x] for x in c] for c in clusters]

In [28]:
sum([len(cl) == 1 for cl in CL])

82

In [29]:
i = 1
with open('../bases/undirected_clusters.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for cl in CL:
        for rome in cl:
           writer.writerow([f'g{i}', rome])
        i += 1

### Directed graph

In [21]:
DG = nx.DiGraph()
DG.add_nodes_from(romes)
weights = {}
R = 1
for rome1 in romes:
    weights[rome1] = {}
    s = 0
    for rome2 in graph[rome1]:
        if True: # replace by rome1[0] = rome2[0]
            weights[rome1][rome2] = np.float_power(data[rome1].corr(data[rome2]),R)
            s += weights[rome1][rome2]
    for rome2 in graph[rome1]:
        w = weights[rome1][rome2]/s
        if w > 0.2: # replace by rome1[0] = rome2[0]
            DG.add_edge(rome1,rome2)

matrix = nx.adjacency_matrix(DG)
result = mc.run_mcl(matrix,inflation=2)
clusters = mc.get_clusters(result)
len(clusters) 

177

In [22]:
CL = [[romes[x] for x in c] for c in clusters]

In [23]:
sum([len(cl) == 1 for cl in CL])

39

In [25]:
i = 1
with open('../bases/directed_clusters.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for cl in CL:
        for rome in cl:
           writer.writerow([f'g{i}', rome])
        i += 1